In [1]:
import pandas as pd
# from sasviya.ml.tree import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
syn_credit_data = pd.read_csv("/workspaces/myfolder/data/generated_lendingClub1M.csv")
len(syn_credit_data)

1000000

In [3]:
syn_credit_data.loc[0]

CreditPolicy                          1
Purpose              debt_consolidation
InterestRate                     0.1265
Installment                      314.98
LogAnnualInc                   11.46961
DebtIncRatio                       8.74
FICOScore                           702
CredLineAge                  4136.84858
RevBalance                         8658
RevUtilization                     75.6
Inquiries6Mnths                       0
Delinquencies2Yrs                     0
PublicRecord                          0
Default                               1
Name: 0, dtype: object

In [4]:
X = syn_credit_data.drop(columns=['Default', 'Purpose'])
y = syn_credit_data['Default']

In [17]:
%%timeit -n 1 -r 1
syn_skgb = GradientBoostingClassifier()
syn_skgb.fit(X, y)

4min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
import sasviya.ml.tree as sasviyaml

In [6]:
sas_gb = sasviyaml.GradientBoostingClassifier()
sas_gb.fit(X, y)

GradientBoostingClassifier()

In [7]:
import sasctl
from pathlib import Path
import os
import requests

In [10]:
## get access token for viya env using refresh token. change to your own viya server and preferred authentication method.
url = "https://apgtps2demo.gtp.unx.sas.com"
auth_url = f"{url}/SASLogon/oauth/token"
## reading long-lived refresh token from txt file
refresh_token = Path(f'/workspaces/{os.environ["DEFAULT_MOUNTNAME"]}/apgtps2demo_refresh_token.txt').read_text().replace('\n', '')

payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': 'Basic c2FzLmNsaTo=',
}

response = requests.request("POST", auth_url, headers=headers, data=payload, verify=False)
# access_token = response.json()['access_token']
response.content

/usr/local/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apgtps2demo.gtp.unx.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


b'<html>\r\n<head><title>503 Service Temporarily Unavailable</title></head>\r\n<body>\r\n<center><h1>503 Service Temporarily Unavailable</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [13]:
st = sasctl.Session("https://apgtps2demo.gtp.unx.sas.com", token=access_token, verify_ssl=False)
st

In [14]:
help(sasctl.register_model)

Help on function register_model in module sasctl.tasks:

register_model(model, name, project, repository=None, input=None, version=None, files=None, force=False, record_packages=True)
    Register a model in the model repository.
    
    Parameters
    ----------
    model : swat.CASTable or sklearn.BaseEstimator
        The model to register.  If an instance of ``swat.CASTable`` the table
        is assumed to hold an ASTORE, which will be downloaded and used to
        construct the model to register.  If a scikit-learn estimator, the
        model will be pickled and uploaded to the registry and score code will
        be generated for publishing the model to MAS.
    name : str
        Designated name for the model in the repository.
    project : str or dict
        The name or id of the project, or a dictionary representation of
        the project.
    repository : str or dict, optional
        The name or id of the repository, or a dictionary representation of
        the repo

In [21]:
from sasctl.services import model_repository as mr, model_management as mm
project = mr.get_project("WB_Workshop")
project.id

'c43204cc-b609-4eee-b052-d5f8940eaa3f'

In [22]:
def register_model(model_name, model, project):
    model_params = {
        "name": model_name,
        "projectId": project.id,
        "type": "ASTORE",
    }
    astore = mr.post(
        "/models",
        files={"files": (f"{model_name}.astore", model.export())},
        data=model_params,
    )
    print(astore)

In [23]:
register_model("MyModel2", sas_gb, project)